<div style="background-color:LightBlue; text-align:center; padding:20px;">
    <h2 style="color:black; font-family: Verdana, sans-serif;"><strong>Reinforcement Learning Project - ViZ Doom</strong></h2>
    <p style="font-size: 14px; color: black; font-family: Verdana, sans-serif;"> 
        <table style="margin: auto; border-collapse: collapse; color: black;">
            <tr>
                <th style="border: 0;">Names</th>
                <th style="border: 0;">GitHub Username</th>
            </tr>
            <tr>
                <td style="border: 0;">Rogier Gernaat</td>
                <td style="border: 0;">RogierHHS</td>
            </tr>
            <tr>
                <td style="border: 0;">Daan Eising</td>
                <td style="border: 0;">DaanEising</td>
            </tr>
            <tr>
                <td style="border: 0;">Julia Boschman</td>
                <td style="border: 0;">JuliaBoschman</td>
            </tr>
            <tr>
                <td style="border: 0;">Jort Kankerhoek</td>
                <td style="border: 0;">Homoboi</td>
            </tr>
        </table>
    </p>
</div>

<div style="display: flex; justify-content: center; align-items: center; margin-top: 10px;">
    <img src="Doomski.png/" alt="Doomski.png" style="width: 1000px; height: auto;">
</div>


- ***Docent***: Vikram Radhakrishnan
- ***Datum***: 08-04-2025


---
<div style="background-color:LightBlue; text-align:center; vertical-align:middle; padding:50px 0; margin-top:5px; margin-bottom:5px">
    <h2 id="eda-title" style="color:black; font-family: Verdana, sans-serif; font-size: 25px;"><strong> Inhoudsopgave </strong></h2>
</div>
<ul style="padding: 0; list-style: none;">
    <li style="text-align: center;"><a href="#1.0">Hoofdstuk 1: Het Project (Use Case)</a></li>
    <li style="text-align: center;"><a href="#2.0">Hoofdstuk 2: Ontwerp en Implementatie</a></li>
    <li style="text-align: center;"><a href="#3.0">Hoofdstuk 3: Valideer en Vergelijk</a></li>
    <li style="text-align: center;"><a href="#4.0">Hoofdstuk 4: Beschrijf je Werk</a></li>
    -
    <li style="text-align: center;"><a href="#5.0">Literatuurlijst</a></li>
</ul>



---
<div style="background-color:LightBlue; text-align:center; vertical-align:middle; padding:50px 0; margin-top:5px; margin-bottom:5px">
    <h2 id="eda-title" style="color:black; font-family: Verdana, sans-serif; font-size: 25px;"><strong> Hoofdstuk 1: Het project (Use Case) </strong></h2>
</div>

### Context en Doel
Dit project richt zich op het ontwikkelen van een Reinforcement Learning (RL) model binnen de VizDoom-omgeving. Het doel is om een agent te trainen die efficiënt leert navigeren en gevechten aan te gaan in de Doom-omgeving.

### Use Case
VizDoom biedt een uitdagende omgeving voor visuele leermodellen, omdat de agent gebruik maakt van visuele input om beslissingen te nemen. RL is geschikt omdat het leert door interactie met de omgeving in plaats van statische gegevens.

### Probleemstelling
Hoe kunnen we een RL-agent trainen om optimaal te presteren in een complexe visuele omgeving zoals VizDoom?



---
<div style="background-color:LightBlue; text-align:center; vertical-align:middle; padding:50px 0; margin-top:5px; margin-bottom:5px">
    <h2 id="eda-title" style="color:black; font-family: Verdana, sans-serif; font-size: 25px;"><strong> Hoofdstuk 2: Ontwerp en Implementatie </strong></h2>
</div>

VizDoom inladen

In [1]:
!pip install vizdoom

     --------------------------------------- 15.5/15.5 MB 12.3 MB/s eta 0:00:00
  Using cached gymnasium-1.1.1-py3-none-any.whl (965 kB)
  Using cached pygame-2.6.1-cp310-cp310-win_amd64.whl (10.6 MB)
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
!cd github & git clone https://github.com/mwydmuch/ViZDoom.git

The system cannot find the path specified.
Cloning into 'ViZDoom'...
Updating files:  19% (398/2057)
Updating files:  20% (412/2057)
Updating files:  21% (432/2057)
Updating files:  22% (453/2057)
Updating files:  23% (474/2057)
Updating files:  24% (494/2057)
Updating files:  25% (515/2057)
Updating files:  26% (535/2057)
Updating files:  27% (556/2057)
Updating files:  28% (576/2057)
Updating files:  29% (597/2057)
Updating files:  30% (618/2057)
Updating files:  31% (638/2057)
Updating files:  32% (659/2057)
Updating files:  33% (679/2057)
Updating files:  34% (700/2057)
Updating files:  35% (720/2057)
Updating files:  36% (741/2057)
Updating files:  37% (762/2057)
Updating files:  38% (782/2057)
Updating files:  39% (803/2057)
Updating files:  40% (823/2057)
Updating files:  41% (844/2057)
Updating files:  42% (864/2057)
Updating files:  43% (885/2057)
Updating files:  44% (906/2057)
Updating files:  45% (926/2057)
Updating files:  46% (947/2057)
Updating files:  47% (967/2057)
Upd

In [2]:
from vizdoom import *
import random
import time

In [11]:
from vizdoom import DoomGame, Mode

game = DoomGame()
game.load_config("ViZDoom/scenarios/basic.cfg")  # laadt de basisconfiguratie incl. basic.wad
game.set_window_visible(True)           # optioneel: render de gamewindow
game.init()

actions = [[1,0,0],  # naar links
           [0,1,0],  # naar rechts
           [0,0,1]]  # schieten

# Uitvoeren van 1 episode met random acties
game.new_episode()
while not game.is_episode_finished():
    state = game.get_state()              # huidige toestand (beeld & variabelen)
    reward = game.make_action(random.choice(actions))
    # ... (hier kun je bijv. je RL-model aansturen i.p.v. random)
score = game.get_total_reward()
print("Score:", score)
game.close()


Score: 95.0



---
<div style="background-color:LightBlue; text-align:center; vertical-align:middle; padding:50px 0; margin-top:5px; margin-bottom:5px">
    <h2 id="eda-title" style="color:black; font-family: Verdana, sans-serif; font-size: 25px;"><strong> Hoofdstuk 3: Valideer en Vergelijk </strong></h2>
</div>


---
<div style="background-color:LightBlue; text-align:center; vertical-align:middle; padding:50px 0; margin-top:5px; margin-bottom:5px">
    <h2 id="eda-title" style="color:black; font-family: Verdana, sans-serif; font-size: 25px;"><strong> Hoofdstuk 4: Beschrijf je Werk </strong></h2>
</div>


---
<div style="background-color:LightBlue; text-align:center; vertical-align:middle; padding:50px 0; margin-top:5px; margin-bottom:5px">
    <h2 id="eda-title" style="color:black; font-family: Verdana, sans-serif; font-size: 25px;"><strong> Literatuurlijst </strong></h2>
</div>